In [1]:
import time
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import load_model # type: ignore

# 1. SETUP
# Load your models
print(" Loading Models...")
# Update these paths if necessary
action_model = load_model('final_sgd_model.h5') 
emotion_model = load_model('Emotions_best2.h5')

def measure_performance(model, input_shape, model_name="Model"):
    print(f"\n MEASURING: {model_name}")
    
    # 2. GENERATE DUMMY DATA
    # Create random noise with the exact shape the model expects
    dummy_input = np.random.rand(*input_shape).astype('float32')
    
    # 3. WARM-UP (Crucial!)
    # The first prediction always takes longer because TensorFlow is compiling the graph.
    # We run 5 dummy predictions to get this out of the way.
    print("    Warming up (ignoring initial lag)...")
    for _ in range(5):
        model.predict(dummy_input, verbose=0)
        
    # 4. TIMING LOOP
    iterations = 100  # Run 100 times to get a good average
    print(f"     Running {iterations} iterations...")
    
    start_time = time.time()
    for _ in range(iterations):
        model.predict(dummy_input, verbose=0)
    end_time = time.time()
    
    # 5. CALCULATE METRICS
    total_time = end_time - start_time
    avg_time_per_prediction = total_time / iterations
    
    latency_ms = avg_time_per_prediction * 1000  # Convert to milliseconds
    fps = 1.0 / avg_time_per_prediction          # Frames Per Second
    
    print(f"      RESULTS for {model_name}:")
    print(f"      Latency: {latency_ms:.2f} ms")
    print(f"      FPS:     {fps:.2f} frames/sec")

# --- RUN TESTS ---

# 1. Test Action Model (Input: 1 batch, 16 frames, 112x112, 3 channels)
measure_performance(
    model=action_model, 
    input_shape=(1, 16, 112, 112, 3), 
    model_name="Action Model (MobileNetV2+LSTM)"
)

# 2. Test Emotion Model (Input: 1 batch, 112x112, 1 channel)
measure_performance(
    model=emotion_model, 
    input_shape=(1, 112, 112, 1), 
    model_name="Emotion Model (ResNet50)"
)

 Loading Models...



 MEASURING: Action Model (MobileNetV2+LSTM)
    Warming up (ignoring initial lag)...
     Running 100 iterations...
      RESULTS for Action Model (MobileNetV2+LSTM):
      Latency: 87.36 ms
      FPS:     11.45 frames/sec

 MEASURING: Emotion Model (ResNet50)
    Warming up (ignoring initial lag)...
     Running 100 iterations...
      RESULTS for Emotion Model (ResNet50):
      Latency: 91.86 ms
      FPS:     10.89 frames/sec
